In [9]:
import torch
import numpy as np

In [10]:
data = [[1,2],[3,4]]
torch.tensor(data)

tensor([[1, 2],
        [3, 4]])

In [11]:
x_np = torch.tensor(np.array([[1,2,3,4],[5,6,7,8]]))
x_np

tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]], dtype=torch.int32)

In [12]:
x_np.size()

torch.Size([2, 4])

In [13]:
x_np.dtype

torch.int32

In [14]:
x_np.device

device(type='cpu')

In [15]:
if torch.cuda.is_available(): #gpu 있나 확인
    x_data = x_np.to('cuda')

In [16]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [17]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [18]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [19]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [20]:
loss_fn = nn.CrossEntropyLoss()

In [21]:
lr = 1e-3

In [22]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)  #Adam 많이 사용 lr은 10^-3 많이 사용

In [23]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # 데이터를 device(gpu있다면 gpu)로 보내기

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312243  [   64/60000]
loss: 0.550135  [ 6464/60000]
loss: 0.389778  [12864/60000]
loss: 0.501902  [19264/60000]
loss: 0.450734  [25664/60000]
loss: 0.433137  [32064/60000]
loss: 0.382276  [38464/60000]
loss: 0.542670  [44864/60000]
loss: 0.487069  [51264/60000]


KeyboardInterrupt: 

In [ ]:
# 모델 저장하기
# torch.save(model,'model.pth') # 모델 전체 저장하기
# torch.save(model.state_dict(), "model.pt") # 모델의 구조가 아닌 학습한 weight만 저장함

In [ ]:
# 모델 불러오기
# model = NeuralNetwork().to(device)
# model.load_state_dict(torch.load("model.pth"))

# 딥러닝 구조
1) 데이터셋 로드
2) 네트워크 구조
3) loss 함수 
4) optimizer
5) train
6) test